# E.T.L (extract-transform-load)


El E.T.L. es el primer paso de todo análisis de datos, se realiza en tres etapas muy bien definidas y que cumplen funciones específicas cuyo prioceso se enumera a continuación: 
1. **Extracción de los datos** relevantes de la base de datos de origen.
2. **Transformación de los datos** para que sean más adecuados para el análisis.
3. **Carga de los datos** en la base de datos de destino.

## Extracción de Datos

En este caso los datos provienen de diversas fuentes, siendo el principal proveedor la API de `Coin Gecko` y `coinMarketCap` a continuación se realiza la extracción de información (Datos) de  Coin Gecko.

In [1]:
# Cargar módulos necesarios
import datetime
import pandas as pd
import numpy as np

from pycoingecko import CoinGeckoAPI

In [2]:
def convert_date(date:any, format:str, to_timestamp = True):
    try:
        if to_timestamp == True:
            date_format = datetime.datetime.strptime(date, format)
            return datetime.datetime.timestamp(date_format)
        elif to_timestamp == False:
            return datetime.datetime.utcfromtimestamp(date).strftime(format)
    except:
        print(f'Ha ocurrido una excepción verifique los datos')
        
    
    

In [3]:
# Cargar datos de MarketCap.csv
global_market = pd.read_csv('datasets/MarketCap.csv', sep=';')
global_market.head(2)

,date,market_cap,market_cap_Billion
0,2018-01-01,"6,89388E+11","0,69"
1,2018-01-02,"7,03326E+11","0,70"


In [4]:
# Verificar estructura de los datos
global_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   date                2059 non-null   object
 1   market_cap          2059 non-null   object
 2   market_cap_Billion  2059 non-null   object
dtypes: object(3)
memory usage: 48.4+ KB


#### Observaciones:
1. Número de columnas: 3
2. Número de filas: 2059
3. Typo de los datos: object(3)
4. Uso de Memoria: 48.4 KB

Se debe realizar trabajo de correcciónde tipo de datos como sigue:
1. `date` de object a datetime
2. `market_cap` y `market_cap_Billion`de object a float

In [5]:
# Convertir datos columna date
global_market['date'] = pd.to_datetime(global_market['date'])
global_market['date'].dtype.name

'datetime64[ns]'

In [6]:
# Convertir datos columnas `market_cap` y `market_cap_Billion`
# Remplazar coma por punto y cambiar tipo a float64
global_market['market_cap'] = global_market['market_cap'].str.replace(',','.').astype('float64')
global_market['market_cap_Billion']=global_market['market_cap_Billion'].str.replace(',','.').astype('float64')
global_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                2059 non-null   datetime64[ns]
 1   market_cap          2059 non-null   float64       
 2   market_cap_Billion  2059 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 48.4 KB


In [7]:
# Renombrar columna market_cap a globa_cap
global_market = global_market.rename(columns={'market_cap':'global_cap', 'market_cap_Billion':'global_cap_Bi'})

# Eliminar filas duplicadas si existen
print(f'Tamaño del set antes de eliminar las filas repetidas: {global_market.shape}')
global_market.drop_duplicates(inplace=True)
print(f'Tamaño del set después de eliminar las filas repetidas: {global_market.shape}')
global_market.head(2)

                     

Tamaño del set antes de eliminar las filas repetidas: (2059, 3)
Tamaño del set después de eliminar las filas repetidas: (2059, 3)


,date,global_cap,global_cap_Bi
0,2018-01-01,6.893880e+11,0.69
1,2018-01-02,7.033260e+11,0.70


In [10]:
# Crear instancia de CoinGeckoAPI
cg = CoinGeckoAPI()

# Crear lista de criptomonedas de interes (Las razones para la escogencia de éstas criptomonedas se explicarán en el EDA.
crypto = ['Bitcoin', 'Ethereum', 'Tether', 'BNB', 'XRP', 'Cardano', 'Render', 
           'Dogecoin', 'Solana', 'TRON', 'Litecoin', 'SingularityNET', 'Fetch.ai']

# Obtener id de criptomonedas.
id_coins = [dic['id'] for dic in cg.get_coins_list() if dic['name'] in crypto]

start_date = '2018-01-01'
end_date = '2023-08-21'
format = '%Y-%m-%d'

In [19]:
# Obtener información de criptos y guardar en archivo .csv
cryptos_fact = pd.DataFrame()
for cry in id_coins:
    try:

        print(f'Nombre: {cry}')
        print('*'*20)
        print()
        # Extraer datos desde la API Coin Gecko
        history =cg.get_coin_market_chart_range_by_id(id=cry, vs_currency = 'usd', from_timestamp = convert_date(start_date, format), 
                                                          to_timestamp=convert_date(end_date, format), localization=False)
        
        # Convertir datos en DataFrame de Pandas y realizar transformaciones neceasrias
        df = pd.DataFrame(history)
        df['date'] = df['prices'].apply(lambda x: convert_date(int(x[0]/1000), format, to_timestamp=False))  
        df['prices'] = df['prices'].apply(lambda x:x[1])
        df['market_caps'] = df['market_caps'].apply(lambda x:x[1])
        df['total_volumes'] = df['total_volumes'].apply(lambda x:x[1])
        
        # Chequeando valores nulos por cada columna en porcentaje
        print('*****  Valores nulos por columna  *****')
        print()
        print(df.isnull().sum()*100/df.shape[0])
        
        # Eliminar filas duplicadas si existen
        print('*****  Verificar existencia de filas duplicadas  *****')
        print()
        print(f'Tamaño del set antes de eliminar las filas repetidas: {df.shape}')
        df.drop_duplicates(inplace=True)
        print(f'Tamaño del set después de eliminar las filas repetidas: {df.shape}')
        print()
        
        # Calcular el rendimiento diario
        df['daily_return'] = df['prices'].pct_change()
        
        # Crear columna name
        df['name'] = cry.title()
        
        # Convertir Columna date in datetime
        df['date'] = pd.to_datetime(df['date'])
        df['date'].dtype.name
        
        # Crear merge entre df y global_market por columna 'date'
        df_merge = pd.merge(df, global_market, how="left", on='date')
        
        # Crear columna de Participación de mercado por cada día
        df_merge['market_share'] = (df_merge['market_caps']/df_merge['global_cap']) * 100
        
        # Reordenamos las columnas y 
        df_merge = df_merge[['date', 'name', 'prices', 'daily_return', 'market_caps', 'global_cap', 'market_share']]
        
        # Guardar información en archivo .csv
        df_merge.to_csv(f'datasets/history_{cry}.csv', index=False, encoding='utf-8')
        print(f'archivo creado con exito y guardado en "datasets/history_{cry}.csv" .....')
        print()
    
        # Concatenar con 'cryptos_fact' para crear tabla de hechos
        cryptos_fact = pd.concat([cryptos_fact, df_merge])
    except:
        print()
        print(f'***** Ha ocurrido una excepción procesando los datos de {cry} *****')
        print()
        continue
        
cryptos_fact.to_csv(f'datasets/crypto_fact_table.csv', index=False, encoding='utf-8')
print('archivo creado con exito y guardado en "datasets/crypto_fact_table.csv" .....')
print('*' * 80 )


Nombre: binancecoin
********************

*****  Valores nulos por columna  *****

prices           0.0
market_caps      0.0
total_volumes    0.0
date             0.0
dtype: float64
*****  Verificar existencia de filas duplicadas  *****

Tamaño del set antes de eliminar las filas repetidas: (2058, 4)
Tamaño del set después de eliminar las filas repetidas: (2058, 4)

archivo creado con exito y guardado en "datasets/history_binancecoin.csv" .....

Nombre: bitcoin
********************

*****  Valores nulos por columna  *****

prices           0.0
market_caps      0.0
total_volumes    0.0
date             0.0
dtype: float64
*****  Verificar existencia de filas duplicadas  *****

Tamaño del set antes de eliminar las filas repetidas: (2058, 4)
Tamaño del set después de eliminar las filas repetidas: (2058, 4)

archivo creado con exito y guardado en "datasets/history_bitcoin.csv" .....

Nombre: cardano
********************

*****  Valores nulos por columna  *****

prices           0.0
market_c

## <span style = 'color: green'> Fin E.T.L.</span>